# Part 1

You need to cross a vast firewall. The firewall consists of several layers, each with a security scanner that moves back and forth across the layer. To succeed, you must not be detected by a scanner.

By studying the firewall briefly, you are able to record (in your puzzle input) the depth of each layer and the range of the scanning area for the scanner within it, written as depth: range. Each layer has a thickness of exactly 1. A layer at depth 0 begins immediately inside the firewall; a layer at depth 1 would start immediately after that.

For example, suppose you've recorded the following:

```
0: 3
1: 2
4: 4
6: 4
```

This means that there is a layer immediately inside the firewall (with range 3), a second layer immediately after that (with range 2), a third layer which begins at depth 4 (with range 4), and a fourth layer which begins at depth 6 (also with range 4). Visually, it might look like this:

```
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]
```

Within each layer, a security scanner moves back and forth within its range. Each security scanner starts at the top and moves down until it reaches the bottom, then moves up until it reaches the top, and repeats. A security scanner takes one picosecond to move one step. Drawing scanners as S, the first few picoseconds look like this:

```
Picosecond 0:
 0   1   2   3   4   5   6
[S] [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

Picosecond 1:
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

Picosecond 2:
 0   1   2   3   4   5   6
[ ] [S] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

Picosecond 3:
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]
```

Your plan is to hitch a ride on a packet about to move through the firewall. The packet will travel along the top of each layer, and it moves at one layer per picosecond. Each picosecond, the packet moves one layer forward (its first move takes it into layer 0), and then the scanners move one step. If there is a scanner at the top of the layer as your packet enters it, you are caught. (If a scanner moves into the top of its layer while you are there, you are not caught: it doesn't have time to notice you before you leave.) If you were to do this in the configuration above, marking your current position with parentheses, your passage through the firewall would look like this:

```
Initial state:
 0   1   2   3   4   5   6
[S] [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

Picosecond 0:
 0   1   2   3   4   5   6
(S) [S] ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
( ) [ ] ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]


Picosecond 1:
 0   1   2   3   4   5   6
[ ] ( ) ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] (S) ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]


Picosecond 2:
 0   1   2   3   4   5   6
[ ] [S] (.) ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] (.) ... [ ] ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]


Picosecond 3:
 0   1   2   3   4   5   6
[ ] [ ] ... (.) [ ] ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]

 0   1   2   3   4   5   6
[S] [S] ... (.) [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]


Picosecond 4:
 0   1   2   3   4   5   6
[S] [S] ... ... ( ) ... [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... ( ) ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]


Picosecond 5:
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] (.) [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [S] ... ... [S] (.) [S]
[ ] [ ]         [ ]     [ ]
[S]             [ ]     [ ]
                [ ]     [ ]


Picosecond 6:
 0   1   2   3   4   5   6
[ ] [S] ... ... [S] ... (S)
[ ] [ ]         [ ]     [ ]
[S]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... ( )
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]
```

In this situation, you are caught in layers 0 and 6, because your packet entered the layer when its scanner was at the top when you entered it. You are not caught in layer 1, since the scanner moved into the top of the layer once you were already there.

The severity of getting caught on a layer is equal to its depth multiplied by its range. (Ignore layers in which you do not get caught.) The severity of the whole trip is the sum of these values. In the example above, the trip severity is `0*3 + 6*4 = 24`.

Given the details of the firewall you've recorded, if you leave immediately, what is the severity of your whole trip?

In [82]:
import textwrap
from io import StringIO

from dataclasses import dataclass, field

In [83]:
@dataclass
class ScannerRange:
    layer: int
    range: int
    pos: int = field(init=False, default=0)
    dir_: str = field(init=False, default='down')
    
    def advance(self):
        if self.dir_ == 'down':
            self.pos += 1
            if self.pos == self.range - 1:
                self.dir_ = 'up'
        else:
            self.pos -= 1
            if self.pos == 0:
                self.dir_ = 'down'
    
    def reset(self):
        self.pos = 0
        self.dir_ = 'down'
    
    def copy(self):
        inst = self.__class__(self.layer, self.range)
        inst.pos = self.pos
        inst.dir_ = self.dir_
        return inst

In [84]:
def init_firewall(rows) -> dict:
    firewall = {}
    for row in rows:
        layer, range = (int(x) for x in row.split(': '))
        firewall[layer] = ScannerRange(layer, range)
    return firewall

In [85]:
def test_input() -> dict:
    return init_firewall(StringIO(textwrap.dedent(
        """\
        0: 3
        1: 2
        4: 4
        6: 4
        """)))

In [86]:
def puzzle_input() -> dict:
    with open('./inputs/day13/input.txt') as f:
        return init_firewall(f)

In [87]:
def one_round(firewall: dict, packet_pos: int) -> int:
    severity = 0
    caught = False
    
    # first check for scanners in this packet position
    if packet_pos in firewall and firewall[packet_pos].pos == 0:
        severity = firewall[packet_pos].range * packet_pos
        caught = True
    
    # then all the scanners move
    for scanner in firewall.values():
        scanner.advance()
    
    return severity, caught

In [88]:
test_firewall = test_input()
assert sum(one_round(test_firewall, pos)[0] for pos in range(max(test_firewall.keys()) + 1)) == 24

In [89]:
puzzle_firewall = puzzle_input()
sum(one_round(puzzle_firewall, pos)[0] for pos in range(max(puzzle_firewall.keys()) + 1))

1504

# Part 2

Now, you need to pass through the firewall without being caught - easier said than done.

You can't control the speed of the packet, but you can delay it any number of picoseconds. For each picosecond you delay the packet before beginning your trip, all security scanners move one step. You're not in the firewall during this time; you don't enter layer 0 until you stop delaying the packet.

In the example above, if you delay 10 picoseconds (picoseconds 0 - 9), you won't get caught:

```
State after delaying:
 0   1   2   3   4   5   6
[ ] [S] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

Picosecond 10:
 0   1   2   3   4   5   6
( ) [S] ... ... [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
( ) [ ] ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]


Picosecond 11:
 0   1   2   3   4   5   6
[ ] ( ) ... ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[S] (S) ... ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]


Picosecond 12:
 0   1   2   3   4   5   6
[S] [S] (.) ... [S] ... [S]
[ ] [ ]         [ ]     [ ]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] (.) ... [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]


Picosecond 13:
 0   1   2   3   4   5   6
[ ] [ ] ... (.) [ ] ... [ ]
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [S] ... (.) [ ] ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]


Picosecond 14:
 0   1   2   3   4   5   6
[ ] [S] ... ... ( ) ... [ ]
[ ] [ ]         [ ]     [ ]
[S]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... ( ) ... [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]


Picosecond 15:
 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] (.) [ ]
[S] [S]         [ ]     [ ]
[ ]             [ ]     [ ]
                [S]     [S]

 0   1   2   3   4   5   6
[S] [S] ... ... [ ] (.) [ ]
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]


Picosecond 16:
 0   1   2   3   4   5   6
[S] [S] ... ... [ ] ... ( )
[ ] [ ]         [ ]     [ ]
[ ]             [S]     [S]
                [ ]     [ ]

 0   1   2   3   4   5   6
[ ] [ ] ... ... [ ] ... ( )
[S] [S]         [S]     [S]
[ ]             [ ]     [ ]
                [ ]     [ ]
```

Because all smaller delays would get you caught, the fewest number of picoseconds you would need to delay to get through safely is 10.

What is the fewest number of picoseconds that you need to delay the packet to pass through the firewall without being caught?

In [90]:
import itertools

## Brute-force Python Solution

In [91]:
def stealth_delay(firewall: dict):
    loop_firewall = {layer: scanner.copy() for layer, scanner in firewall.items()}
    num_layers = max(firewall.keys())
    
    for delay in itertools.count(0):
        pre_check_firewall = {layer: scanner.copy() for layer, scanner in loop_firewall.items()}
        
        # check for safe passage
        if all(
                not one_round(loop_firewall, pos)[1]
                for pos in range(num_layers + 1)):
            return delay
        else:
            # reset to pre-check state and advance once
            loop_firewall = {layer: scanner.copy() for layer, scanner in pre_check_firewall.items()}
            for scanner in loop_firewall.values():
                scanner.advance()

In [92]:
assert stealth_delay(test_input()) == 10, stealth_delay(test_firewall)

In [93]:
%time stealth_delay(puzzle_input())

CPU times: user 10min 28s, sys: 1.92 s, total: 10min 30s
Wall time: 10min 33s


3823370

## Brute-force Cython Solution

In [1]:
import textwrap
from io import StringIO

In [2]:
def test_input() -> dict:
    rows = StringIO(textwrap.dedent(
        """\
        0: 3
        1: 2
        4: 4
        6: 4
        """))
    return {layer: range_ for layer, range_ in (map(int, row.strip().split(': ')) for row in rows)}

In [3]:
test_input()

{0: 3, 1: 2, 4: 4, 6: 4}

In [3]:
def puzzle_input() -> dict:
    with open('./inputs/day13/input.txt') as f:
        return {layer: range_ for layer, range_ in (map(int, row.strip().split(': ')) for row in f)}

In [4]:
%load_ext cython

In [9]:
%%cython
cpdef int scanner_layer(int scanner_range, int t):
    cdef int sr = scanner_range - 1
    cdef int scx2 = sr * 2
    cdef int t_mod_scx2 = t % scx2  
    return t_mod_scx2 if t_mod_scx2 <= sr else scx2 - (t % scx2)

In [10]:
assert [scanner_layer(3, t) for t in range(7)] == [0, 1, 2, 1, 0, 1, 2]

In [11]:
[scanner_layer(6, t) for t in range(20)]

[0, 1, 2, 3, 4, 5, 4, 3, 2, 1, 0, 1, 2, 3, 4, 5, 4, 3, 2, 1]

In [5]:
%%cython
cdef int scanner_layer(int scanner_range, int t):
    cdef int sr = scanner_range - 1
    cdef int scx2 = sr * 2
    cdef int t_mod_scx2 = t % scx2  
    return t_mod_scx2 if t_mod_scx2 <= sr else scx2 - t_mod_scx2

cdef bint attempt_crossing(dict firewall, int num_layers, int t_start):
    cdef int pos, scanner_pos, scanner_range
    for pos in range(num_layers):
        if pos in firewall:
            scanner_range = firewall[pos]
            scanner_pos = scanner_layer(scanner_range, t_start + pos)
            if scanner_pos == 0:
                return False
    return True

cpdef int find_start(dict firewall):
    cdef int t_start = 0
    cdef int num_layers = max(firewall.keys()) + 1
    while True:
        if attempt_crossing(firewall, num_layers, t_start):
            break
        else:
            t_start += 1
    return t_start

In [13]:
assert find_start(test_input()) == 10

In [7]:
puzzle_firewall = puzzle_input()

In [17]:
%timeit find_start(puzzle_firewall)

547 ms ± 59.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Even More Cythonic

In [9]:
import array

In [10]:
def firewall_array(firewall: dict):
    ranges = []
    for i in range(max(firewall.keys()) + 1):
        ranges.append(firewall[i] if i in firewall else 0)
    return array.array('i', ranges)

In [11]:
%%cython
cimport cython

cdef int scanner_layer2(int scanner_range, int t) nogil:
    cdef int sr = scanner_range - 1
    cdef int scx2 = sr * 2
    cdef int t_mod_scx2 = t % scx2
    return t_mod_scx2 if t_mod_scx2 <= sr else scx2 - t_mod_scx2

@cython.boundscheck(False)
@cython.wraparound(False)
cdef bint attempt_crossing2(int[:] ranges, int num_layers, int t_start) nogil:
    cdef int pos, scanner_pos, scanner_range
    for pos in range(num_layers):
        if ranges[pos] != 0:
            scanner_pos = scanner_layer2(ranges[pos], t_start + pos)
            if scanner_pos == 0:
                return False
    return True

@cython.boundscheck(False)
@cython.wraparound(False)
cpdef int find_start2(int[:] ranges):
    cdef int t_start = 0
    cdef int num_layers = len(ranges)
    while True:
        if attempt_crossing2(ranges, num_layers, t_start):
            break
        else:
            t_start += 1
    return t_start

In [13]:
ranges = firewall_array(test_input())
ranges

array('i', [3, 2, 0, 0, 4, 0, 4])

In [14]:
assert find_start2(ranges) == 10

In [15]:
puzzle_ranges = firewall_array(puzzle_input())

In [18]:
%timeit find_start2(puzzle_ranges)

52.6 ms ± 1.62 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Numba

In [19]:
import numpy as np
from numba import boolean, int32, jit

In [20]:
@jit(int32(int32, int32), nopython=True)
def py_scanner_layer(scanner_range, t):
    sr = scanner_range - 1
    scx2 = sr * 2
    t_mod_scx2 = t % scx2  
    return t_mod_scx2 if t_mod_scx2 <= sr else scx2 - t_mod_scx2


@jit(boolean(int32[:], int32, int32), nopython=True)
def py_attempt_crossing(ranges, num_layers, t_start):
    for pos in range(num_layers):
        if ranges[pos] != 0:
            scanner_pos = py_scanner_layer(ranges[pos], t_start + pos)
            if scanner_pos == 0:
                return False
    return True


@jit(int32(int32[:]), nopython=True)
def py_find_start(ranges):
    t_start = 0
    num_layers = len(ranges)
    while True:
        if py_attempt_crossing(ranges, num_layers, t_start):
            break
        else:
            t_start += 1
    return t_start

In [21]:
assert py_find_start(np.int32(firewall_array(test_input()))) == 10

In [22]:
puzzle_ranges = np.int32(firewall_array(puzzle_input()))

In [23]:
%timeit py_find_start(puzzle_ranges)

70.3 ms ± 3.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Cython Algorithm but Plain Python

In [22]:
def py_scanner_layer(scanner_range: int, t: int) -> int:
    sr = scanner_range - 1
    scx2 = sr * 2
    t_mod_scx2 = t % scx2  
    return t_mod_scx2 if t_mod_scx2 <= sr else scx2 - t_mod_scx2

def py_attempt_crossing(firewall: dict, num_layers: int, t_start: int) -> bool:
    for pos in range(num_layers):
        if pos in firewall:
            scanner_range = firewall[pos]
            scanner_pos = py_scanner_layer(scanner_range, t_start + pos)
            if scanner_pos == 0:
                return False
    return True

def py_find_start(firewall: dict) -> int:
    t_start = 0
    num_layers = max(firewall.keys()) + 1
    while True:
        if py_attempt_crossing(firewall, num_layers, t_start):
            break
        else:
            t_start += 1
    return t_start

In [23]:
assert py_find_start(test_input()) == 10

In [24]:
%time py_find_start(puzzle_input())

CPU times: user 6.54 s, sys: 41.1 ms, total: 6.58 s
Wall time: 6.64 s


3823370